# 📚 Hệ gợi ý Memory-based: Item-based CF (Cosine + Mean-centering + Shrinkage)

Notebook này hiện thực **Item-based Collaborative Filtering** theo hướng memory-based:
- Chuẩn hoá ID → chỉ số 0..n-1 (gộp cả train & test)
- Ma trận rating thưa **R (user × item)**
- **Mean-centering** theo từng user: $\tilde{r}_{ui} = r_{ui} - \mu_u$
- **Cosine similarity** giữa cột item trên ma trận đã center
- **Shrinkage** theo số user đồng-rating: $s' = \frac{n_{co}}{n_{co}+\alpha} s$ trong đó $n_{co}$ là số người dùng đã đánh giá cả 2 item i và j
- Dự đoán: $\hat{r}_{ui} = \mu_u + \frac{\sum_j s'(i,j)\,(r_{uj}-\mu_u)}{\sum_j |s'(i,j)|}$ (top-K láng giềng)
- Xuất file **submission** dạng `Id,Score` (Id = 1..N theo thứ tự test)


In [14]:

import numpy as np
import pandas as pd
from scipy import sparse
from pathlib import Path

# --- Đường dẫn (sửa nếu cần) ---
TRAIN_PATH = "data/train.txt"   # ví dụ: "/kaggle/input/.../train.txt"
TEST_PATH  = "data/test.txt"    # ví dụ: "/kaggle/input/.../test.txt"
SUB_PATH   = "output/submission_itemCF.csv"

print(Path(TRAIN_PATH).resolve())
print(Path(TEST_PATH).resolve())


E:\Sao lưu onedrive\not de ra truong nao\Hệ gợi ý\Project_RS_in_HUST\data\train.txt
E:\Sao lưu onedrive\not de ra truong nao\Hệ gợi ý\Project_RS_in_HUST\data\test.txt


In [15]:

# --- Đọc dữ liệu cách bởi khoảng trắng/tab ---
df_train = pd.read_csv(
    TRAIN_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid","rating"],
    engine="python",
)
df_test = pd.read_csv(
    TEST_PATH,
    sep=r"\s+",
    header=None,
    names=["userid","movieid"],
    engine="python",
)
print(df_train.shape, df_test.shape)
display(df_train.head())
display(df_test.head())


(90570, 3) (9430, 2)


,userid,movieid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


,userid,movieid
0,1,20
1,1,33
2,1,61
3,1,117
4,1,155


In [16]:

# --- Ánh xạ ID gốc -> chỉ số liên tục 0..n-1 (gộp cả train & test để không thiếu id) ---
all_u = pd.concat([df_train["userid"], df_test["userid"]], ignore_index=True)
all_i = pd.concat([df_train["movieid"], df_test["movieid"]], ignore_index=True)

uid_uniques = all_u.drop_duplicates()
iid_uniques = all_i.drop_duplicates()

uid2idx = pd.Series(np.arange(len(uid_uniques), dtype=np.int32), index=uid_uniques.values)
iid2idx = pd.Series(np.arange(len(iid_uniques), dtype=np.int32), index=iid_uniques.values)

df_train["u_idx"] = df_train["userid"].map(uid2idx).astype(np.int32)
df_train["i_idx"] = df_train["movieid"].map(iid2idx).astype(np.int32)
df_test["u_idx"]  = df_test["userid"].map(uid2idx).astype(np.int32)
df_test["i_idx"]  = df_test["movieid"].map(iid2idx).astype(np.int32)

n_users = uid2idx.size
n_items = iid2idx.size
print(f"n_users={n_users}, n_items={n_items}, train_rows={len(df_train)}, test_rows={len(df_test)}")


n_users=943, n_items=1682, train_rows=90570, test_rows=9430


In [17]:

# --- Tạo ma trận thưa R (user × item) và mean-center theo từng user ---
rows = df_train["u_idx"].to_numpy()
cols = df_train["i_idx"].to_numpy()
vals = df_train["rating"].astype(np.float32).to_numpy()

R = sparse.csr_matrix((vals, (rows, cols)), shape=(n_users, n_items))

# Trung bình theo user (mu_u). Nếu user chưa có rating -> dùng global mean
user_sum = np.array(R.sum(axis=1)).ravel()
user_cnt = np.diff(R.indptr)  # số phần tử khác 0 mỗi dòng
global_mean = float(vals.mean()) if len(vals) else 3.0
user_mean = np.divide(
    user_sum,
    user_cnt,
    out=np.full_like(user_sum, global_mean, dtype=np.float64),
    where=user_cnt != 0
)

# R_centered = R - mu_u theo từng dòng
R_centered = R.tocsr(copy=True).astype(np.float32)
for u in range(n_users):
    s, e = R_centered.indptr[u], R_centered.indptr[u+1]
    if s < e:
        R_centered.data[s:e] -= user_mean[u]

# Chuyển sang CSC để thao tác theo cột (item) nhanh
X = R_centered.tocsc()
print("Matrix ready:", R.shape, "nnz:", R.nnz)


Matrix ready: (943, 1682) nnz: 90570


In [18]:
def predict_ui_item_based(u, i, K=50, alpha=20.0, clip_min=1.0, clip_max=5.0):
    # Các item J mà user u đã có rating
    s, e = R.indptr[u], R.indptr[u+1]
    J = R.indices[s:e]
    if J.size == 0:
        # fallback: trung bình người dùng (hoặc global_mean nếu cần)
        r_hat = user_mean[u] if np.isfinite(user_mean[u]) else global_mean
        return float(np.clip(r_hat, clip_min, clip_max))

    # Cột i (đã center) và các cột tương ứng item J
    xi = X[:, i]
    if xi.nnz == 0:
        r_hat = user_mean[u] if np.isfinite(user_mean[u]) else global_mean
        return float(np.clip(r_hat, clip_min, clip_max))
    XJ = X[:, J]

    # Cosine similarity (chuyển kết quả về mảng 1D)
    num = (xi.T @ XJ).toarray().ravel()
    xi_norm = np.sqrt(xi.multiply(xi).sum())
    XJ_norm = np.sqrt(np.array((XJ.multiply(XJ)).sum(axis=0)).ravel())
    denom = xi_norm * XJ_norm
    sim = np.divide(num, denom, out=np.zeros_like(num), where=denom != 0)

    # Shrinkage: s' = n_co/(n_co+alpha) * s  (chia an toàn)
    xi_bin = (xi != 0).astype(np.int8)
    XJ_bin = (XJ != 0).astype(np.int8)
    n_co = (xi_bin.T @ XJ_bin).toarray().ravel()
    shrink = np.divide(n_co, n_co + alpha, out=np.zeros_like(n_co, dtype=float), where=(n_co + alpha) != 0)
    sim = sim * shrink

    # Top-K theo |sim|
    if J.size > K:
        topk_idx = np.argpartition(np.abs(sim), -K)[-K:]
        J_top   = J[topk_idx]
        sim_top = sim[topk_idx]
    else:
        J_top   = J
        sim_top = sim

    # (r_uj - μ_u) và công thức suy luận
    r_u_centered = R_centered[u, J_top].toarray().ravel()
    denom2 = float(np.sum(np.abs(sim_top)))

    # Nếu không có hàng xóm hợp lệ -> fallback
    if not np.isfinite(denom2) or denom2 < 1e-12:
        r_hat = user_mean[u] if np.isfinite(user_mean[u]) else global_mean
        return float(np.clip(r_hat, clip_min, clip_max))

    delta = float(np.sum(sim_top * r_u_centered) / denom2)
    if not np.isfinite(delta):
        r_hat = user_mean[u] if np.isfinite(user_mean[u]) else global_mean
        return float(np.clip(r_hat, clip_min, clip_max))

    r_hat = user_mean[u] + delta
    return float(np.clip(r_hat, clip_min, clip_max))


In [19]:
K = 50
ALPHA = 20.0

preds = np.array(
    [predict_ui_item_based(u, i, K=K, alpha=ALPHA)
     for u, i in zip(df_test["u_idx"].to_numpy(), df_test["i_idx"].to_numpy())],
    dtype=np.float64
)

# Đảm bảo mọi giá trị đều hữu hạn (không NaN/Inf)
finite_mask = np.isfinite(preds)
if not finite_mask.all():
    safe_mean = np.nanmean(preds[finite_mask]) if finite_mask.any() else float(global_mean)
    preds[~finite_mask] = safe_mean

# (tùy) clip về [1,5]
preds = np.clip(preds, 1.0, 5.0)

submission = pd.DataFrame({
    "Id": np.arange(1, len(preds) + 1, dtype=np.int64),
    "Score": preds
})

# Ghi file — dùng float_format để tránh ký tự rỗng kỳ quặc
submission.to_csv(SUB_PATH, index=False, float_format="%.6f")
print("Đã lưu:", SUB_PATH, submission.shape, submission.dtypes)


Đã lưu: output/submission_itemCF.csv (9430, 2) Id         int64
Score    float64
dtype: object
